<img src="imagenes/rn3.png" width="200">
<img src="http://www.identidadbuho.uson.mx/assets/letragrama-rgb-150.jpg" width="200">

# [Curso de Redes Neuronales](https://curso-redes-neuronales-unison.github.io/Temario/)

# Una red neuronal multicapa simple usando TensorFlow


[**Julio Waissman Vilanova**](http://mat.uson.mx/~juliowaissman/), 27 de septiembre de 2017.



En esta libreta se muestra el ejemplo básico para una red multicapa sencilla
aplicada al conjunto de datos [MNIST](http://yann.lecun.com/exdb/mnist/).

Esta libreta es básicamente una traducción del ejemplo
desarrollado por [Aymeric Damien](https://github.com/aymericdamien/TensorFlow-Examples/)


In [1]:
import tensorflow as tf

## 1. Cargar datos

Primero cargamos los archivos que se utilizan para el aprendizaje. Para otro tipo de problemas, es necesario hacer un proceso conocido como *Data Wrangling*, que normalmente se realiza con la ayuda de *Pandas*. 

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

Para que un aprendizaje tenga sentido es necesario tener bien separado un conjunto de datos de aprendizaje y otro de prueba (en caso de grandes conjuntos de datos es la opción). Como vemos tanto las imágenes como las etiquetas están separados en archivos de datos y de aprendizaje.

El objeto `mnist` es un objeto tensorflow que contiene 3 objetos tipo tensorflow: *test*, *train* y *validation*, los cuales a su vez contienen *ndarrays* de *numpy*. La estructura es la misma para cada conjunto de datos. Veamos su estructura:


In [3]:
print("Tipo de images: {}".format(type(mnist.train.images)))
print("Tipo de epochs_completed: {}".format(type(mnist.train.epochs_completed)))
print("Tipo de labels: {}".format(type(mnist.train.labels)))
print("Tipo de nest_batch: {}".format(type(mnist.train.next_batch)))
print("Tipo de num_examples: {}".format(type(mnist.train.num_examples)))

Tipo de images: <class 'numpy.ndarray'>
Tipo de epochs_completed: <class 'int'>
Tipo de labels: <class 'numpy.ndarray'>
Tipo de nest_batch: <class 'method'>
Tipo de num_examples: <class 'int'>


Como generar el conjunto de datos para ser utilizado dentro de TensorFlow es objeto de otra libreta. Por el momento concentremonos en como hacer una red neuronal rápido y sin dolor.

Sin embargo, vamos a ver unos cuantos datos que nos pueden ser de útilidad para la construcción de la red neuronal.

In [4]:
print("Forma del ndarray con las imágenes: {}".format(mnist.train.images.shape))
print("Forma del ndarray con las etiquetas: {}".format(mnist.train.labels.shape))
print("-" * 79)
print("Número de imagenes de entrenamiento: {}".format(mnist.train.images.shape[0]))
print("Tamaño de las imagenes: {}".format(mnist.train.images.shape[1]))
print("Clases diferentes: {}".format(mnist.train.labels.shape[1]))


Forma del ndarray con las imágenes: (55000, 784)
Forma del ndarray con las etiquetas: (55000, 10)
-------------------------------------------------------------------------------
Número de imagenes de entrenamiento: 55000
Tamaño de las imagenes: 784
Clases diferentes: 10


## 2. Construcción de la red neuronal

Para hacer una red neuronal lo más genérica posible y que pdamos reutilizar en otros proyectos, vamos a establecer los parámetros base independientemente de la inicialización de la red, independientemente de la forma en que construimos la red. 

Comencemos por establecer una función genérica que nos forme una red neuronal con dos capas ocultas. No agrego más comentarios porque, con la experiencia de las libretas anteriores, la construcción de la red neuronal se explica sola.

In [5]:
def red_neuronal_dos_capas_ocultas(x, pesos, sesgos):
    """
    Genera una red neuronal de dos capas para usar en TensorFlow
    
    Parámetros
    ----------
    pesos: un diccionario con tres etiquetas: 'h1', 'h2' y 'ho'
           en donde cada una es una tf.Variable conteniendo una 
           matriz de dimensión [num_neuronas_capa_anterior, num_neuronas_capa]
                  
    sesgos: un diccionario con tres etiquetas: 'b1', 'b2' y 'bo'
            en donde cada una es una tf.Variable conteniendo un
            vector de dimensión [numero_de_neuronas_capa]
                   
    Devuelve
    --------
    Un ops de tensorflow que calcula la salida de una red neuronal
    con dos capas ocultas, y activaciones RELU.
    
    """
    # Primera capa oculta con activación ReLU
    capa_1 = tf.matmul(x, pesos['h1'])
    capa_1 = tf.add(capa_1, sesgos['b1'])
    capa_1 = tf.nn.relu(capa_1)
    
    # Segunda capa oculta con activación ReLU
    capa_2 = tf.matmul(capa_1, pesos['h2'])
    capa_2 = tf.add(capa_2, sesgos['b2'])
    capa_2 = tf.nn.relu(capa_2)
    
    # Capa de salida con activación lineal
    # En Tensorflow la salida es siempre lineal, y luego se especifica
    # la función de salida a la hora de calcularla como vamos a ver 
    # más adelante
    capa_salida = tf.matmul(capa_2, pesos['ho']) + sesgos['bo']
    return capa_salida

Y ahora necesitamos poder generar los datos de entrada a la red neuronal de
alguna manera posible. Afortunadamente sabemos exactamente que necesitaos, así
que vamos a hacer una función que nos genere las variables de peso y sesgo.

Por el momento, y muy a la brava, solo vamos a generarlas con números aletorios con una 
distribución $\mathcal{N}(0, 1)$.

In [6]:
def inicializa_pesos(entradas, n1, n2, salidas):
    """
    Genera un diccionario con pesos  
    para ser utilizado en la función red_neuronal_dos_capas_ocultas
    
    Parámetros
    ----------
    entradas: Número de neuronas en la capa de entrada
    
    n1: Número de neuronas en la primer capa oculta
    
    n2: Número de neuronas en la segunda capa oculta
    
    salidas: Número de neuronas de salida
    
    Devuelve
    --------
    Dos diccionarios, uno con los pesos por capa y otro con los sesgos por capa
    
    """
    pesos = {
        'h1': tf.Variable(tf.random_normal([entradas, n1])),
        'h2': tf.Variable(tf.random_normal([n1, n2])),
        'ho': tf.Variable(tf.random_normal([n2, salidas]))
    }
    
    sesgos = {
        'b1': tf.Variable(tf.random_normal([n1])),
        'b2': tf.Variable(tf.random_normal([n2])),
        'bo': tf.Variable(tf.random_normal([salidas]))
    }
    
    return pesos, sesgos

Ahora necesitamos establecer los parámetros de la topología de la red neuronal. 
Tomemos en cuenta que estos prámetros los podríamos haber establecido desde
la primer celda, si el fin es estar variando los parámetros para escoger los que 
ofrezcan mejor desempeño.

In [7]:
num_entradas = 784  #  Lo sabemos por la inspección que hicimos a mnist
num_salidas = 10    # Ídem

# Aqui es donde podemos jugar
num_neuronas_capa_1 = 256
num_neuronas_capa_2 = 256

¡A construir la red! Para esto vamos a necesitar crear las entradas
con un placeholder, y crear nuestra topología de red neuronal.

Observa que la dimensión de x será [None, num_entradas], lo que significa que 
la cantidad de renglones es desconocida (o variable).

In [8]:
# La entrada a la red neuronal
x = tf.placeholder("float", [None, num_entradas])

# Los pesos y los sesgos
w, b = inicializa_pesos(num_entradas, num_neuronas_capa_1, num_neuronas_capa_2, num_salidas)

# Crea la red neuronal
estimado = red_neuronal_dos_capas_ocultas(x, w, b)


Instructions for updating:
Colocations handled automatically by placer.


Parecería que ya está todo listo. Sin ambargo falta algo muy importante: No hemos explicado
ni cual es el criterio de error (loss) que vamos a utilizar, ni cual va a ser el método de
optimización (aprendizaje) que hemos decidido aplicar.

Primero definamos el costo que queremos minimizar, y ese costo va a estar en función de lo
estimado con lo real, por lo que necesitamos otra entrada de datos para los datos de salida.

Sin ningun lugar a dudas, el costo que mejor describe este problema es el de *softmax*


In [9]:
#  Creamos la variable de datos de salida conocidos
y = tf.placeholder("float", [None, num_salidas])

#  Definimos la función de costo
costo = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=estimado, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Y ahora definimos que función de aprendizaje vamos a utilizar. Existen muchas funciones
de aprendizaje en tensorflow, las cuales se pueden consultar en `tf.train.`. Entre las
existentes podemos ver algunas conocidas del curso como descenso de gradiente simple,
momento, rprop, rmsprop entre otras. Casi todas las funciones de optimización (aprendizaje)
acaban su nombre con `Optimize`.

En este caso vamos a usar un método comocido como el *algoritmo de Adam* el cual 
se puede consultar [aqui](http://arxiv.org/pdf/1412.6980.pdf). El metodo utiliza dos calculos
de momentos diferentes, y por lo visto genera resultados muy interesantes desde el punto 
de vista práctico.

¿Cual es el mejor método? Pues esto es en función de tu problema y de la cantidad de datos que tengas.
Lo mejor es practicar con varios métodos para entender sus ventajas y desventajas.

En todo caso el método de optimización requiere que se le inicialice con una tasa de aprendizaje.

In [10]:
alfa = 0.001
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)


## 3. Ejecutar la sesión usando mini-batches

Ahora, ya que la red neuronal está lista vamos a ejecutar la red utilizando el algoritmo de
Adam pero en forma de mini-batches. Con el fin de tener control sobre el problema, vamos a establecer un número máximo de epoch (ciclos de aprendizaje), el tamaño de los mini-batches, y cada cuandos epoch 
quisieramos ver como está evolucionando la red neuronal.

Como entrenar una red neuronal no tiene sentido, si no es porque la queremos usar para reconocer,
no tendría sentido entrenarla y luego perderla y tener que reentrenar en cada ocasión. Recuerda que cuando
se cierra la sesión se borra todo lo que se tenía en memoria. 

Para esto vamos a usar una ops especial llamada `Saver`, que permite guardar en un archivo la red neuronal y 
después utilizarla en otra sesión (en otro script, computadora, ....).


In [11]:
archivo_modelo = "rnn2.ckpt"
saver = tf.train.Saver()

Como todo se ejecuta dentro de una sesión, no es posible hacerlo por partes (si usamos el 
`with` que debería ser la única forma en la que iniciaramos una sesión). Por lo tanto procuraré dejar comentado el código.

In [12]:
numero_epochs = 30
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

Epoch: 0            Costo: 160.83733494151724
Epoch: 1            Costo: 41.71173904765732
Epoch: 2            Costo: 26.339589040062645
Epoch: 3            Costo: 18.597581534385686
Epoch: 4            Costo: 13.631360631659858
Epoch: 5            Costo: 10.124610042969206
Epoch: 6            Costo: 7.723124431487444
Epoch: 7            Costo: 5.741367376846774
Epoch: 8            Costo: 4.3009156647635205
Epoch: 9            Costo: 3.2330499522703855
Epoch: 10           Costo: 2.5650033133265335
Epoch: 11           Costo: 1.92610898914332
Epoch: 12           Costo: 1.4684379593541275
Epoch: 13           Costo: 1.1741105068786926
Epoch: 14           Costo: 0.9336911004276206
Epoch: 15           Costo: 0.7696978474093763
Epoch: 16           Costo: 0.7385285291160316
Epoch: 17           Costo: 0.57963557503229
Epoch: 18           Costo: 0.5225207725525219
Epoch: 19           Costo: 0.5253588119058014
Epoch: 20           Costo: 0.49752028947470495
Epoch: 21           Costo: 0.45253640538

Ahora vamos a revisar que tan bien realizó el aprendizaje cuando se aplica la red adatos que
no se usaron para entrenamiento. Para esto vamos a utilizar dos ops extas: una 
para definir la operaración de datos bien estimados o mal estimados, y otra para
calcular el promedio de datos bien estimados. Para calcular los datos bien estimados vamos a utilizar `tf.cast` que permite ajustar los tipos
al tipo tensor. 

In [13]:
prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))



Ahora si, vamos a abrir una nueva sesión, vamos a restaurar los valores de la sesión anterior,
y vamos a ejecutar el grafo con el fin de evaluar la ops precision, pero ahora con el
diccionario de alimentación con los datos de prueba.

In [14]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from rnn2.ckpt
Precisión: 0.9584000110626221


## 4. Contesta las siguientes preguntas

1. ¿Que pasa si aumenta el número de epochs? ¿Cuando deja de ser util aumentar los epoch?  
  Va aumentando la precisión, deja de ser util cuando comienza a bajar, que en mis pruebas fue al rededor de 175 epochs

2. ¿Que pasa si aumentas o disminuyes la tasa de aprendizaje?  
Aumentando la tasa de aprendizaje el costo disminuye más rapido, sin embargo, si es muy alta, deja de converger dando una precisión muy mala.
Disminuyendo la taza de aprendizaje converge más despacio.


3. Utiliza al menos otros 2 métodos de optimización (existentes en Tensorflow), ajustalos y comparalos. ¿Cual de los métodos te gusta más y porque preferirías unos sobre otros?  
El AdamOptimizer me dio el mejor resultado en el mejor tiempo.

4. ¿Que pasa si cambias el tamaño de los minibatches?
5. ¿Como harías si dejaste a medias un proceso de aprendizaje (en 10 epochs por ejemplo) y quisieras entrenar la red 10 epoch más, y mañana quisieras entrenarla otros 10 epoch más?  
Guardar el modelo.

**Para contestar las preguntas, agrega cuantas celdas con comentarios y con códgo sean necesarias.** Aprovecha que las libretas de *Jupyter* te permite hacerte una especie de tutorial personalizado.

In [19]:
#Incrementando el número de epochs
alfa = 0.001
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "rnn2.ckpt"
saver = tf.train.Saver()

numero_epochs = 175
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 162.79438453674314
Epoch: 1            Costo: 40.19226976047864
Epoch: 2            Costo: 25.616192274527116
Epoch: 3            Costo: 17.930013914325006
Epoch: 4            Costo: 13.239954518784167
Epoch: 5            Costo: 9.847955660340991
Epoch: 6            Costo: 7.508212439054253
Epoch: 7            Costo: 5.580312634984546
Epoch: 8            Costo: 4.2541547291619
Epoch: 9            Costo: 3.1981649510210306
Epoch: 10           Costo: 2.4617553194316883
Epoch: 11           Costo: 2.0157561061705187
Epoch: 12           Costo: 1.4198508812438277
Epoch: 13           Costo: 1.1831781622287671
Epoch: 14           Costo: 0.9883424979616886
Epoch: 15           Costo: 0.7602920162358346
Epoch: 16           Costo: 0.6982391424481048
Epoch: 17           Costo: 0.6395305333745769
Epoch: 18           Costo: 0.5517198173148719
Epoch: 19           Costo: 0.40668263780887476
Epoch: 20           Costo: 0.4975058415454841
Epoch: 21           Costo: 0.35614539019

Precisión: 0.9731000065803528


In [21]:
#Aumentando la taza de aprendizaje
alfa = 0.1
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "rnn2.ckpt"
saver = tf.train.Saver()

numero_epochs = 175
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 40.43866363969715
Epoch: 1            Costo: 0.9745015419071368
Epoch: 2            Costo: 0.8024116824973706
Epoch: 3            Costo: 0.7408012174476276
Epoch: 4            Costo: 0.5359023708917882
Epoch: 5            Costo: 0.5150971717726105
Epoch: 6            Costo: 0.4755817582932387
Epoch: 7            Costo: 0.46872062595053104
Epoch: 8            Costo: 0.46760605950247147
Epoch: 9            Costo: 0.6404999000646853
Epoch: 10           Costo: 0.801818022023548
Epoch: 11           Costo: 1.041679102940992
Epoch: 12           Costo: 0.9574362411282279
Epoch: 13           Costo: 1.0237107291004868
Epoch: 14           Costo: 1.0763780826872054
Epoch: 15           Costo: 1.0426527708226982
Epoch: 16           Costo: 1.2230848617987193
Epoch: 17           Costo: 1.1402769866856661
Epoch: 18           Costo: 1.3711064869707281
Epoch: 19           Costo: 1.4034507898850876
Epoch: 20           Costo: 1.405176920999181
Epoch: 21           Costo: 1.4551691

In [22]:
#Reduciendo la taza de aprendizaje
alfa = 0.0005
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "rnn2.ckpt"
saver = tf.train.Saver()

numero_epochs = 175
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 254.21471038818396
Epoch: 1            Costo: 62.56794752120975
Epoch: 2            Costo: 41.21759744990956
Epoch: 3            Costo: 30.350189034071814
Epoch: 4            Costo: 23.468277419697124
Epoch: 5            Costo: 18.791855746108034
Epoch: 6            Costo: 15.0309694515846
Epoch: 7            Costo: 12.2764116409421
Epoch: 8            Costo: 9.971785438114955
Epoch: 9            Costo: 8.19853761590441
Epoch: 10           Costo: 6.714460858339727
Epoch: 11           Costo: 5.564960572212129
Epoch: 12           Costo: 4.523332721494287
Epoch: 13           Costo: 3.646768335725474
Epoch: 14           Costo: 2.954550225110011
Epoch: 15           Costo: 2.425433627400534
Epoch: 16           Costo: 1.898528239601207
Epoch: 17           Costo: 1.557473380023218
Epoch: 18           Costo: 1.2693301802143944
Epoch: 19           Costo: 1.0002131830044076
Epoch: 20           Costo: 0.7379625281962043
Epoch: 21           Costo: 0.6028432024959061
Epoch

Se acabaron los epochs, saliendo de la sesión de tensorflow.
INFO:tensorflow:Restoring parameters from rnn2.ckpt
Precisión: 0.9628000259399414


In [26]:
#Método 1
alfa = 0.6
optimizador = tf.train.ProximalAdagradOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "rnn2.ckpt"
saver = tf.train.Saver()

numero_epochs = 175
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 280.005361230156
Epoch: 1            Costo: 2.023594398715279
Epoch: 2            Costo: 1.8592671244794656
Epoch: 3            Costo: 1.7169317117604348
Epoch: 4            Costo: 1.5895839383385393
Epoch: 5            Costo: 1.5300214145400308
Epoch: 6            Costo: 1.4773481858860358
Epoch: 7            Costo: 1.4348641239513054
Epoch: 8            Costo: 1.3779966463825921
Epoch: 9            Costo: 1.3510505399920723
Epoch: 10           Costo: 1.3020347636396232
Epoch: 11           Costo: 1.2733697082779616
Epoch: 12           Costo: 1.2043682751872313
Epoch: 13           Costo: 1.17810735247352
Epoch: 14           Costo: 1.127011257626793
Epoch: 15           Costo: 1.0957499301433564
Epoch: 16           Costo: 1.0646249361471694
Epoch: 17           Costo: 1.057452653429726
Epoch: 18           Costo: 0.9956239119443014
Epoch: 19           Costo: 0.9840698994289749
Epoch: 20           Costo: 0.9605577254295347
Epoch: 21           Costo: 0.963667759570

In [27]:
#Método 2
alfa = 0.6
optimizador = tf.train.AdadeltaOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "rnn2.ckpt"
saver = tf.train.Saver()

numero_epochs = 175
tamano_minibatch = 100
display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 370.94552099054516
Epoch: 1            Costo: 104.45338495081121
Epoch: 2            Costo: 71.81907029585412
Epoch: 3            Costo: 55.93650263006032
Epoch: 4            Costo: 46.732541037472835
Epoch: 5            Costo: 40.10797598752108
Epoch: 6            Costo: 35.9858842225508
Epoch: 7            Costo: 30.73039533755996
Epoch: 8            Costo: 28.954560588489883
Epoch: 9            Costo: 25.61892104343932
Epoch: 10           Costo: 24.023249946074053
Epoch: 11           Costo: 21.414994392936887
Epoch: 12           Costo: 19.696618300134492
Epoch: 13           Costo: 18.199477016536342
Epoch: 14           Costo: 17.29295752855865
Epoch: 15           Costo: 16.036629317944705
Epoch: 16           Costo: 14.973961072618293
Epoch: 17           Costo: 14.06086731344461
Epoch: 18           Costo: 12.834988652711568
Epoch: 19           Costo: 12.525297011984165
Epoch: 20           Costo: 11.485162926610268
Epoch: 21           Costo: 11.0887034464479

Se acabaron los epochs, saliendo de la sesión de tensorflow.
INFO:tensorflow:Restoring parameters from rnn2.ckpt
Precisión: 0.9336000084877014


In [29]:
#Cambiando el tama;o de los minichatch
alfa = 0.001
optimizador = tf.train.AdamOptimizer(learning_rate=alfa)
paso_entrenamiento = optimizador.minimize(costo)

archivo_modelo = "rnn2.ckpt"
saver = tf.train.Saver()

numero_epochs = 175
tamano_minibatch = 30

display_step = 1

# Muy importante la primera vez que se ejecuta inicializar todas las variables
init = tf.global_variables_initializer()

# La manera correcta de iniciar una sesión y realizar calculos
with tf.Session() as sess:
    sess.run(init)

    # Ciclos de entrenamiento
    for epoch in range(numero_epochs):

        #  Inicializa el costo promedio de todos los minibatches en 0
        avg_cost = 0.
        
        #  Calcula el número de minibatches que se pueden usar 
        total_batch = int(mnist.train.num_examples/tamano_minibatch)

        #  Por cada minibatch
        for i in range(total_batch):
            
            #  Utiliza un generador incluido en mnist que obtiene 
            #  tamano_minibatch ejemplos selecionados aleatoriamente del total
            batch_x, batch_y = mnist.train.next_batch(tamano_minibatch)
            
            #  Ejecuta la ops del paso_entrenamiento para aprender 
            #  y la del costo, con el fin de mostrar el aprendizaje
            _, c = sess.run([paso_entrenamiento, costo], feed_dict={x: batch_x, y: batch_y})
            
            #  Calcula el costo del minibatch y lo agrega al costo total
            avg_cost += c / total_batch
        
        # Muestra los resultados
        if epoch % display_step == 0:
            print (("Epoch: " + str(epoch)).ljust(20)
                   + ("Costo: " + str(avg_cost)))
    
    #  Guarda la sesión en el archivo rnn2.cptk
    saver.save(sess, archivo_modelo)
    
    print("Se acabaron los epochs, saliendo de la sesión de tensorflow.")

prediction_correcta = tf.equal(tf.argmax(estimado, 1), tf.argmax(y, 1))

precision = tf.reduce_mean(tf.cast(prediction_correcta, "float"))

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, archivo_modelo)
    porcentaje_acierto = sess.run(precision, feed_dict={x: mnist.test.images,
                                                        y: mnist.test.labels})
    print("Precisión: {}".format(porcentaje_acierto))

Epoch: 0            Costo: 100.5326173076838
Epoch: 1            Costo: 24.293468637316053
Epoch: 2            Costo: 13.89282890653465
Epoch: 3            Costo: 9.051084436480894
Epoch: 4            Costo: 5.875730512371206
Epoch: 5            Costo: 3.9480447181074205
Epoch: 6            Costo: 2.4665073531256128
Epoch: 7            Costo: 2.2419377844511645
Epoch: 8            Costo: 1.5858777416452772
Epoch: 9            Costo: 1.2279949018589396
Epoch: 10           Costo: 1.0745202171916584
Epoch: 11           Costo: 0.8865324347168612
Epoch: 12           Costo: 0.8450870968488302
Epoch: 13           Costo: 0.697859825610904
Epoch: 14           Costo: 0.7329831353162285
Epoch: 15           Costo: 0.6770642213092567
Epoch: 16           Costo: 0.6164504264123338
Epoch: 17           Costo: 0.5308579060216483
Epoch: 18           Costo: 0.5517203094027436
Epoch: 19           Costo: 0.557607201105532
Epoch: 20           Costo: 0.5494761097596081
Epoch: 21           Costo: 0.44420352057

INFO:tensorflow:Restoring parameters from rnn2.ckpt
Precisión: 0.9761999845504761
